Practicumopdrachten Week 4.3 (eerste kans)

Dependecies

In [4]:
import pandas as pd
import pyodbc
import sqlite3

Connectie met smss database

In [5]:
DB = {
    'servername': '(local)\\SQLEXPRESS',
    'database': 'DEDS_DataWarehouse',}

export_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + ';DATABASE=' + DB['database'] + ';Trusted_Connection=yes')

export_cursor = export_conn.cursor()

# check if connection is successful, else throw an error
if export_conn:
    print("Connection with database is established")
else:
    print("Connection with database is not established")
    #raise Exception("Connection with database is not established")

ProgrammingError: ('42000', '[42000] [Microsoft][ODBC SQL Server Driver][SQL Server]Cannot open database "DEDS_DataWarehouse" requested by the login. The login failed. (4060) (SQLDriverConnect); [42000] [Microsoft][ODBC SQL Server Driver][SQL Server]Cannot open database "DEDS_DataWarehouse" requested by the login. The login failed. (4060)')

# Brontabellen
importeer de brontabellen uit de brondata foler

In [6]:
# Connectie met sqlite databases
go_crm_connection = sqlite3.connect('brondata/go_crm.sqlite')
go_sales_connection = sqlite3.connect('brondata/go_sales.sqlite')
go_staff_connection = sqlite3.connect('brondata/go_staff.sqlite')


In [7]:
# inlezen csv bestanden
go_sales_inventory_levels = pd.read_csv('brondata/GO_SALES_INVENTORY_LEVELSData.csv')
go_sales_product_forcast = pd.read_csv('brondata/GO_SALES_PRODUCT_FORECASTData.csv')

## Inlezen van de sqlite tabellen

### go_crm

In [8]:
go_crm_age_group = pd.read_sql_query("SELECT * FROM age_group", go_crm_connection)
go_crm_country = pd.read_sql_query("SELECT * FROM country", go_crm_connection)
go_crm_retailer = pd.read_sql_query("SELECT * FROM retailer", go_crm_connection)
go_crm_retailer_contact = pd.read_sql_query("SELECT * FROM retailer_contact", go_crm_connection)
go_crm_retailer_headquarters = pd.read_sql_query("SELECT * FROM retailer_headquarters", go_crm_connection)
go_crm_retailer_segment = pd.read_sql_query("SELECT * FROM retailer_segment", go_crm_connection)
go_crm_retailer_site = pd.read_sql_query("SELECT * FROM retailer_site", go_crm_connection)
go_crm_retailer_type = pd.read_sql_query("SELECT * FROM retailer_type", go_crm_connection)
go_crm_sales_demographic = pd.read_sql_query("SELECT * FROM sales_demographic", go_crm_connection)
go_crm_sales_territory = pd.read_sql_query("SELECT * FROM sales_territory", go_crm_connection)

### go_sales

In [9]:
go_sales_country = pd.read_sql_query("SELECT * FROM country", go_sales_connection)
go_sales_order_details = pd.read_sql_query("SELECT * FROM order_details", go_sales_connection)
go_sales_order_header = pd.read_sql_query("SELECT * FROM order_header", go_sales_connection)
go_sales_order_method = pd.read_sql_query("SELECT * FROM order_method", go_sales_connection)
go_sales_product = pd.read_sql_query("SELECT * FROM product", go_sales_connection)
go_sales_product_line = pd.read_sql_query("SELECT * FROM product_line", go_sales_connection)
go_sales_product_type = pd.read_sql_query("SELECT * FROM product_type", go_sales_connection)
go_sales_retailer_site = pd.read_sql_query("SELECT * FROM retailer_site", go_sales_connection)
go_sales_return_reason = pd.read_sql_query("SELECT * FROM return_reason", go_sales_connection)
go_sales_returned_item = pd.read_sql_query("SELECT * FROM returned_item", go_sales_connection)
go_sales_sales_branch = pd.read_sql_query("SELECT * FROM sales_branch", go_sales_connection)
go_sales_sales_staff = pd.read_sql_query("SELECT * FROM sales_staff", go_sales_connection)

### go_staff

In [10]:
go_staff_course = pd.read_sql_query("SELECT * FROM course", go_staff_connection)
go_staff_sales_branch = pd.read_sql_query("SELECT * FROM sales_branch", go_staff_connection)
go_staff_sales_staff = pd.read_sql_query("SELECT * FROM sales_staff", go_staff_connection)
go_staff_satisfaction = pd.read_sql_query("SELECT * FROM satisfaction", go_staff_connection)
go_staff_satisfaction_type = pd.read_sql_query("SELECT * FROM satisfaction_type", go_staff_connection)
go_staff_training = pd.read_sql_query("SELECT * FROM training", go_staff_connection)

# Combineren van de tabellen
Combineren van de brondata om tot ons ETL te komen

| Returned_item |
| --- |
| RETURNED_ITEM_code |
| RETURNED_ITEM_DATE_date |
| RETURNED_ITEM_QUANTITY_quantity |
| RETURNED_ITEM_ORDER_DETAIL_CODE_detail_code |
| RETURNED_ITEM_RETURN_REASON_code |
| RETURNED_ITEM_RETURN_REASON_desctiption_en |
| RETURNED_ITEM_RETURNED_ITEMS_TOTAL_PRICE_total |

| Unit |
| --- |
| UNIT_id |
| UNIT_COST_cost |
| UNIT_PRICE_price |
| UNIT_SALE_sale |

| Sales_staff |
| --- |
| SALES_STAFF__code |
| SALES_STAFF_email |
| SALES_STAFF_extension |
| SALES_STAFF_POSITION_EN_position |
| SALES_STAFF_WORK_PHONE_work_phone |
| SALES_STAFF_DATE_HIRED_hired |
| SALES_STAFF_MANAGER_CODE_manager |
| SALES_STAFF_FAX_fax |
| SALES_STAFF_FIRST_NAME_first_name |
| SALES_STAFF_LAST_NAME_last_name |
| SALES_STAFF_FULL_NAME__full_name |
| SALES_STAFF_SALES_BRANCH_CODE_branch_code |
| SALES_STAFF_SALES_BRANCH_ADDRESS_address |
| SALES_STAFF_SALES_BRANCH_ADDRESS_EXTRA_address_extra |

| Satisfaction_type |
| --- |
| SATISFACTION_TYPE_code |
| SATISFACTION_TYPE_description |
| SATISFACTION_TYPE_description_short |

| Course |
| --- |
| COURSE_code |
| COURSE_description |
| COURSE_description_short |

| Year |
| --- |
| YEAR |

| Date            | 
|-----------------|
| DATE_date       |
| DATE_YEAR_year  |
| DATE_MONTH_month |
| DATE_DAY_day |

| Order |
| --- |
| ORDER_order_number |
| ORDER_ORDER_METHOD_CODE_method_code |
| ORDER_ORDER_METHOD_EN_method |

| Retailer_site |
| --- |
| RETAILER_SITE_code |
| RETAILER_SITE_COUNTRY_CODE_country |
| RETAILER_SITE_CITY_city |
| RETAILER_SITE_REGION_region |
| RETAILER_SITE_POSTAL_ZONE_postal_zone |
| RETAILER_SITE_RETAILER_CODE_retailer_code |
| RETAILER_SITE_ACTIVE_INDICATOR_indicator |
| RETAILER_SITE_ADDRESS1_address |
| RETAILER_SITE_ADDRESS2_address |
| RETAILER_SITE_MAIN_ADDRESS__address |

| Sales_branch |
| --- |
| SALES_BRANCH_code |
| SALES_BRANCH_COUNTRY_CODE_country |
| SALES_BRANCH_REGION_region |
| SALES_BRANCH_CITY_city |
| SALES_BRANCH_POSTAL_ZONE_postal_zone |
| SALES_BRANCH_ADDRESS1_address |
| SALES_BRANCH_ADDRESS2_address |
| SALES_BRANCH_MAIN_ADDRESS_address |

| Retailer_contact |
| --- |
| RETAILER_CONTACT_code |
| RETAILER_CONTACT_email |
| RETAILER_CONTACT_RETAILER_SITE_CODE_site_code |
| RETAILER_CONTACT_JOB_POSITION_EN_position |
| RETAILER_CONTACT_EXTENSION_extension |
| RETAILER_CONTACT_FAX_fax |
| RETAILER_CONTACT_GENDER_gender |
| RETAILER_CONTACT_FIRST_NAME_first_name |
| RETAILER_CONTACT_LAST_NAME_last_name |
| RETAILER_CONTACT_FULL_NAME_full_name |

| Retailer |
| --- |
| RETAILER_code |
| RETAILER__name |
| RETAILER_COMPANY_CODEMR_company |
| RETAILER_RETAILER_TYPE_retailer_type_code |
| RETAILER_RETAILER_TYPE_retailer_type_EN |

| Product |
| --- |
| PRODUCT_number |
| PRODUCT_name_product |
| PRODUCT_description_description |
| PRODUCT_image_image |
| PRODUCT_INTRODUCTION_DATE_introduced |
| PRODUCT_PRODUCTION_COST_cost |
| PRODUCT_MARGIN_margin |
| PRODUCT_LANGUAGE_language |
| PRODUCT_MINIMUM_SALE_PRICE_minPrice |
| PRODUCT_PRODUCT_LINE_code |
| PRODUCT_PRODUCT_LINE_code_en |
| PRODUCT_PRODUCT_TYPE_code |
| PRODUCT_PRODUCT_TYPE_code_en |

| Order_details |
| --- |
| ORDER_DETAILS_code |
| ORDER_DETAILS_QUANTITY_quantity |
| ORDER_DETAILS_TOTAL_COST_total |
| ORDER_DETAILS_TOTAL_MARGIN_margin |
| ORDER_DETAILS_RETURN_CODE_returned |
| ORDER_DETAILS_ORDER_NUMBER_order |
| ORDER_DETAILS_PRODUCT_NUMBER_product |
| ORDER_DETAILS_UNIT_ID_unit |

| SALES_TARGETData |
| --- |
| SALES_TARGETDATA_id |
| SALES_TARGETDATA_SALES_YEAR |
| SALES_TARGETDATA_SALES_PERIOD |
| SALES_TARGETDATA_RETAILER_NAME |
| SALES_TARGETDATA_SALES_TARGET |
| SALES_TARGETDATA_TARGET_COST |
| SALES_TARGETDATA_TARGET_MARGIN |
| SALES_TARGETDATA_SALES_STAFF_CODE |
| SALES_TARGETDATA_PRODUCT_NUMBER |
| SALES_TARGETDATA_RETAILER_CODE |

| Training |
| --- |
| TRAINING_SALES_STAFF_CODE |
| TRAINING_COURSE_CODE |
| TRAINING_YEAR |

| Satisfaction |
| --- |
| SATISFACTION_SALES_STAFF_CODE |
| SATISFACTION_SATISFACTION_TYPE_CODE |
| SATISFACTION_YEAR |

| Order_header |
| --- |
| ORDER_HEADER_number |
| ORDER_HEADER_RETAILER_NAME |
| ORDER_HEADER_SALES_STAFF |
| ORDER_HEADER_SALES_BRANCH_CODE |
| ORDER_HEADER_ORDER_DATE |
| ORDER_HEADER_RETAILER_SITE_CODE |
| ORDER_HEADER_RETAILER_CONTACT_CODE |

| GO_SALES_INVENTORY_LEVELS |
| --- |
| GO_SALES_INVENTORY_LEVELS_id |
| GO_SALES_INVENTORY_LEVELS_INVENTORY_COUNT |
| GO_SALES_INVENTORY_LEVELS_PRODUCT_NUMBER |
| GO_SALES_INVENTORY_LEVELS_YEAR_MONTH |

| GO_SALES_PRODUCT_FORECAST |
| --- |
| GO_SALES_PRODUCT_FORECAST_id |
| GO_SALES_PRODUCT_FORECAST_EXPECTED_VOLUME |
| GO_SALES_PRODUCT_FORECAST_EXPECTED_COST |
| GO_SALES_PRODUCT_FORECAST_EXPECTED_MARGIN |
| GO_SALES_PRODUCT_FORECAST_PRODUCT_NUMBER |
| GO_SALES_PRODUCT_FORECAST_YEAR_MONTH |

### returned_item

In [11]:
# to be used in order details
returned_item_columns = ['RETURN_CODE', 'RETURN_DATE', 'RETURN_QUANTITY', 'ORDER_DETAIL_CODE', 'RETURN_REASON_CODE', 'RETURN_DESCRIPTION_EN']
returned_item = pd.merge(go_sales_returned_item, go_sales_return_reason, left_on='RETURN_REASON_CODE', right_on='RETURN_REASON_CODE')
returned_item = pd.merge(returned_item, go_sales_order_details, left_on='ORDER_DETAIL_CODE', right_on='ORDER_DETAIL_CODE')

# filtering the columns
returned_item = returned_item[returned_item_columns]

# renaming the columns
returned_item = returned_item.rename(columns={'RETURN_CODE': 'RETURNED_ITEM_code', 'RETURN_DATE': 'RETURNED_ITEM_DATE_date', 'RETURN_QUANTITY': 'RETURNED_ITEM_QUANTITY_quantity', 'ORDER_DETAIL_CODE': 'RETURNED_ITEM_ORDER_DETAIL_CODE_detail_code', 'RETURN_REASON_CODE': 'RETURNED_ITEM_RETURN_REASON_code', 'RETURN_DESCRIPTION_EN': 'RETURNED_ITEM_RETURN_REASON_desctiption_en'})

print(returned_item.head())

  RETURNED_ITEM_code RETURNED_ITEM_DATE_date RETURNED_ITEM_QUANTITY_quantity  \
0               1491       1-8-2021 04:10:24                               8   
1               1492      6-12-2020 18:46:19                               2   
2               1493      24-6-2021 09:23:14                              22   
3               1494      21-7-2021 00:00:09                              20   
4               1496      7-10-2020 05:13:58                               2   

  RETURNED_ITEM_ORDER_DETAIL_CODE_detail_code  \
0                                       84858   
1                                       84440   
2                                       84867   
3                                       84873   
4                                       84488   

  RETURNED_ITEM_RETURN_REASON_code RETURNED_ITEM_RETURN_REASON_desctiption_en  
0                                5                     Unsatisfactory product  
1                                2                         Incom

### Unit

In [12]:
unit_columns = ['UNIT_COST', 'UNIT_PRICE', 'UNIT_SALE_PRICE']
unit = go_sales_order_details
# make unit_id the primary key, autoincrement
unit['UNIT_ID'] = range(1, len(unit) + 1)
unit = unit.set_index('UNIT_ID')

# filtering the columns
unit = unit[unit_columns]

# renaming the columns
unit = unit.rename(columns={'UNIT_COST': 'UNIT_COST_cost', 'UNIT_PRICE': 'UNIT_PRICE_price', 'UNIT_SALE_PRICE': 'UNIT_SALE_sale'})

print(unit.head())

        UNIT_COST_cost UNIT_PRICE_price UNIT_SALE_sale
UNIT_ID                                               
1                 16.1            22.54          22.54
2                 16.1            22.54          22.54
3                 16.1            22.54          22.54
4                 16.1            22.54          22.54
5                 16.1            22.54          22.54


### Sales_staff

In [13]:
sales_staff_columns = ['SALES_STAFF_CODE', 'EMAIL', 'EXTENSION', 'POSITION_EN', 'WORK_PHONE', 'DATE_HIRED', 'MANAGER_CODE', 'FAX', 'FIRST_NAME', 'LAST_NAME',  'ADDRESS1', 'ADDRESS2']

sales_staff = pd.merge(go_staff_sales_staff, go_staff_sales_branch, left_on='SALES_BRANCH_CODE', right_on='SALES_BRANCH_CODE')

# filtering the columns
sales_staff = sales_staff[sales_staff_columns]

# renaming the columns
sales_staff = sales_staff.rename(columns={'SALES_STAFF_CODE': 'SALES_STAFF_code', 'EMAIL': 'SALES_STAFF_email', 'EXTENSION': 'SALES_STAFF_extension', 'POSITION_EN': 'SALES_STAFF_POSITION_EN_position', 'WORK_PHONE': 'SALES_STAFF_WORK_PHONE_work_phone', 'DATE_HIRED': 'SALES_STAFF_DATE_HIRED_hired', 'MANAGER_CODE': 'SALES_STAFF_MANAGER_CODE_manager', 'FAX': 'SALES_STAFF_FAX_fax', 'FIRST_NAME': 'SALES_STAFF_FIRST_NAME_first_name', 'LAST_NAME': 'SALES_STAFF_LAST_NAME_last_name', 'ADDRESS1': 'SALES_STAFF_SALES_BRANCH_ADDRESS_address', 'ADDRESS2': 'SALES_STAFF_SALES_BRANCH_ADDRESS_EXTRA_address_extra'})

print(sales_staff.head())

  SALES_STAFF_code        SALES_STAFF_email SALES_STAFF_extension  \
0              100  TSavolainen@grtd123.com                   825   
1              101         CKim@grtd123.com                  1228   
2              102        JChoi@grtd123.com                  2641   
3              103        KKwao@grtd123.com                  1541   
4              104         CFon@grtd123.com                  2311   

  SALES_STAFF_POSITION_EN_position SALES_STAFF_WORK_PHONE_work_phone  \
0     Level 2 Sales Representative               +358(0)17 - 433 127   
1     Level 2 Sales Representative                     82-2-778-6587   
2     Level 3 Sales Representative                     82-2-778-6587   
3     Level 2 Sales Representative                   886-2-2975-1988   
4     Level 3 Sales Representative                   886-2-2975-1988   

  SALES_STAFF_DATE_HIRED_hired SALES_STAFF_MANAGER_CODE_manager  \
0      23-Jul-1998 12:00:00 AM                               18   
1      15-Dec-1998

### Satisfaction_type

In [14]:
satisfaction_type = go_staff_satisfaction_type

#dropping trial633 column
satisfaction_type = satisfaction_type.drop(columns=['TRIAL633'])

# rename the columns
satisfaction_type = satisfaction_type.rename(columns={'SATISFACTION_TYPE_CODE': 'SATISFACTION_TYPE_code', 'DESCRIPTION': 'SATISFACTION_TYPE_description'})

print(satisfaction_type.head())

  SATISFACTION_TYPE_code SATISFACTION_TYPE_DESCRIPTION
0                      1                 Not satisfied
1                      2           Less than satisfied
2                      3                     Satisfied
3                      4                Very Satisfied
4                      5           More than satisfied


### Course

In [15]:
course = go_staff_course

# dropping the trial633 column
course = course.drop(columns=['TRIAL633'])

# rename the columns
course = course.rename(columns={'COURSE_CODE': 'COURSE_code', 'DESCRIPTION': 'COURSE_description'})

print(course.head())

  COURSE_code COURSE_DESCRIPTION
0           1     GO Orientation
1           2   GO Communication
2           3         GO Sales 1
3           4         GO Sales 2
4           5     GO Marketing 1


### Year

In [16]:
# to be used in training and satisfaction
# making year dataframe with year as the primary key
year = pd.DataFrame({'YEAR': pd.date_range(start='1/1/1990', end='1/1/2200').year})

# adding all the years from the satisfaction and training data.
year = pd.concat([year, pd.DataFrame({'YEAR': go_staff_training['YEAR']})])
year = pd.concat([year, pd.DataFrame({'YEAR': go_staff_satisfaction['YEAR']})])

# drop duplicates
year = year.drop_duplicates()

# setting the index
year = year.set_index('YEAR')

print(year.head())

Empty DataFrame
Columns: []
Index: [1990, 1991, 1992, 1993, 1994]


### Date

In [17]:
# to be used in order header
date = pd.DataFrame({'DATE_date': pd.date_range(start='1/1/1990', end='1/1/2200')})

# adding all the years from the order header data
date = pd.concat([date, pd.DataFrame({'DATE_date': go_sales_order_header['ORDER_DATE']})])

# drop duplicates
date = date.drop_duplicates()

# setting the index
date['DATE_date'] = pd.to_datetime(date['DATE_date']) # convert to datetime
date = date.set_index('DATE_date')

print(date.head())

Empty DataFrame
Columns: []
Index: [1990-01-01 00:00:00, 1990-01-02 00:00:00, 1990-01-03 00:00:00, 1990-01-04 00:00:00, 1990-01-05 00:00:00]


### Order

In [18]:
# to be used in order header
# making order dataframe with order_number as the primary key 
order = pd.DataFrame({'ORDER_order_number': range(1, len(go_sales_order_method) + 1)})
order['ORDER_ORDER_METHOD_CODE_method_code'] = go_sales_order_method['ORDER_METHOD_CODE']
order['ORDER_ORDER_METHOD_EN_method'] = go_sales_order_method['ORDER_METHOD_EN']

order = order.set_index('ORDER_order_number')

print(order.head())

                   ORDER_ORDER_METHOD_CODE_method_code  \
ORDER_order_number                                       
1                                                    1   
2                                                    2   
3                                                    3   
4                                                    4   
5                                                    5   

                   ORDER_ORDER_METHOD_EN_method  
ORDER_order_number                               
1                                           Fax  
2                                     Telephone  
3                                          Mail  
4                                        E-mail  
5                                           Web  


### Retailer_site

In [19]:
retailer_site = go_sales_retailer_site

# dropping the trial888 column
retailer_site = retailer_site.drop(columns=['TRIAL888'])

# rename the columns
retailer_site = retailer_site.rename(columns={'RETAILER_SITE_CODE': 'RETAILER_SITE_code', 'COUNTRY_CODE': 'RETAILER_SITE_COUNTRY_CODE_country', 'CITY': 'RETAILER_SITE_CITY_city', 'REGION': 'RETAILER_SITE_REGION_region', 'POSTAL_ZONE': 'RETAILER_SITE_POSTAL_ZONE_postal_zone', 'RETAILER_CODE': 'RETAILER_SITE_RETAILER_CODE_retailer_code', 'ACTIVE_INDICATOR': 'RETAILER_SITE_ACTIVE_INDICATOR_indicator', 'ADDRESS1': 'RETAILER_SITE_ADDRESS1_address', 'ADDRESS2': 'RETAILER_SITE_ADDRESS2_address'})

print(retailer_site.head())

  RETAILER_SITE_code RETAILER_SITE_RETAILER_CODE_retailer_code  \
0                  1                                        89   
1                 10                                        92   
2                100                                        95   
3                101                                        95   
4                102                                       108   

     RETAILER_SITE_ADDRESS1_address RETAILER_SITE_ADDRESS2_address  \
0                1117 Franklin Blvd                           None   
1  Suite 1500, 802 Docklands Street                           None   
2                    4830 Princeton                         Unit 2   
3                  6630 Burnet Road                           None   
4              2189 Louisville Road                           None   

  RETAILER_SITE_CITY_city RETAILER_SITE_REGION_region  \
0                Winnipeg                    Manitoba   
1               Vancouver            British Columbia   
2          

### Sales_branch

In [20]:
sales_branch = go_sales_sales_branch

# dropping the trial888 column
sales_branch = sales_branch.drop(columns=['TRIAL888'])

# rename the columns
sales_branch = sales_branch.rename(columns={'SALES_BRANCH_CODE': 'SALES_BRANCH_code', 'COUNTRY_CODE': 'SALES_BRANCH_COUNTRY_CODE_country', 'REGION': 'SALES_BRANCH_REGION_region', 'CITY': 'SALES_BRANCH_CITY_city', 'POSTAL_ZONE': 'SALES_BRANCH_POSTAL_ZONE_postal_zone', 'ADDRESS1': 'SALES_BRANCH_ADDRESS1_address', 'ADDRESS2': 'SALES_BRANCH_ADDRESS2_address', 'MAIN_ADDRESS': 'SALES_BRANCH_MAIN_ADDRESS_address'})

print(sales_branch.head())

  SALES_BRANCH_code SALES_BRANCH_ADDRESS1_address  \
0                13                Schwabentor 35   
1                14              Leopoldstraße 36   
2                15           Isafjordsgatan 30 C   
3                17  7800, 756 - 6th Avenue. S.W.   
4                18              789 Yonge Street   

  SALES_BRANCH_ADDRESS2_address SALES_BRANCH_CITY_city  \
0                          None                Hamburg   
1                          None                München   
2                          None                  Kista   
3                          None                Calgary   
4                          None                Toronto   

  SALES_BRANCH_REGION_region SALES_BRANCH_POSTAL_ZONE_postal_zone  \
0                       None                              D-22529   
1                       None                              D-81241   
2                       None                             S-164 40   
3                    Alberta                            

### Retailer_contact

In [21]:
retailer_contact = go_crm_retailer_contact

# dropping the trial222 column
retailer_contact = retailer_contact.drop(columns=['TRIAL222'])

# rename the columns
retailer_contact = retailer_contact.rename(columns={'RETAILER_CONTACT_CODE': 'RETAILER_CONTACT_code', 'E_MAIL': 'RETAILER_CONTACT_email', 'RETAILER_SITE_CODE': 'RETAILER_CONTACT_RETAILER_SITE_CODE_site_code', 'JOB_POSITION_EN': 'RETAILER_CONTACT_JOB_POSITION_EN_position', 'EXTENSION': 'RETAILER_CONTACT_EXTENSION_extension', 'FAX': 'RETAILER_CONTACT_FAX_fax', 'GENDER' : 'RETAILER_CONTACT_GENDER_gender', 'FIRST_NAME': 'RETAILER_CONTACT_FIRST_NAME_first_name', 'LAST_NAME': 'RETAILER_CONTACT_LAST_NAME_last_name'})

print(retailer_contact.head())

  RETAILER_CONTACT_code RETAILER_CONTACT_RETAILER_SITE_CODE_site_code  \
0                    10                                            14   
1                   100                                            85   
2                   101                                            84   
3                   102                                            88   
4                   103                                            92   

  RETAILER_CONTACT_FIRST_NAME_first_name RETAILER_CONTACT_LAST_NAME_last_name  \
0                                  Frank                                Smith   
1                                 Amanda                                Yates   
2                                  Paula                              Schmidt   
3                                Micheal                                  Tao   
4                                   Jack                               Murphy   

  RETAILER_CONTACT_JOB_POSITION_EN_position  \
0                          

### Retailer

In [22]:
retailer = pd.merge(go_crm_retailer, go_crm_retailer_type, left_on='RETAILER_TYPE_CODE', right_on='RETAILER_TYPE_CODE')

# dropping the trial219 and trial222 columns
retailer = retailer.drop(columns=['TRIAL219', 'TRIAL222'])

# rename the columns
retailer = retailer.rename(columns={'RETAILER_CODE': 'RETAILER_code', 'RETAILER_NAME': 'RETAILER_name', 'COMPANY_CODEMR': 'RETAILER_COMPANY_CODEMR_company', 'RETAILER_TYPE_CODE': 'RETAILER_RETAILER_TYPE_retailer_type_code', 'RETAILER_TYPE_EN': 'RETAILER_RETAILER_TYPE_retailer_type_EN'})

print(retailer.head())

  RETAILER_code RETAILER_CODEMR                COMPANY_NAME  \
0             1            None                   ActiForme   
1           100              70                Golf Masters   
2           101              71             The Marketplace   
3           102              72    Camping Equipment Online   
4           103              73  Tamarack Outfitter Rentals   

  RETAILER_RETAILER_TYPE_retailer_type_code  \
0                                         5   
1                                         1   
2                                         2   
3                                         3   
4                                         5   

  RETAILER_RETAILER_TYPE_retailer_type_EN  
0                  Equipment Rental Store  
1                               Golf Shop  
2                        Department Store  
3                        Direct Marketing  
4                  Equipment Rental Store  


### Product

In [23]:
product = pd.merge(go_sales_product, go_sales_product_type, left_on='PRODUCT_TYPE_CODE', right_on='PRODUCT_TYPE_CODE')
product = pd.merge(product, go_sales_product_line, left_on='PRODUCT_LINE_CODE', right_on='PRODUCT_LINE_CODE')

#dropping the trial888 column
product = product.drop(columns=['TRIAL888'])

# rename the columns
product = product.rename(columns={'PRODUCT_NUMBER': 'PRODUCT_number', 'PRODUCT_NAME': 'PRODUCT_name_product', 'DESCRIPTION': 'PRODUCT_description_description', 'PRODUCT_IMAGE': 'PRODUCT_image_image', 'INTRODUCTION_DATE': 'PRODUCT_INTRODUCTION_DATE_introduced', 'PRODUCTION_COST': 'PRODUCT_PRODUCTION_COST_cost', 'MARGIN': 'PRODUCT_MARGIN_margin', 'LANGUAGE': 'PRODUCT_LANGUAGE_language',  'PRODUCT_TYPE_CODE': 'PRODUCT_PRODUCT_TYPE_code', 'PRODUCT_TYPE_EN': 'PRODUCT_PRODUCT_TYPE_code_en', 'PRODUCT_LINE_CODE': 'PRODUCT_PRODUCT_LINE_code', 'PRODUCT_LINE_EN': 'PRODUCT_PRODUCT_LINE_code_en'})

print(product.head())

  PRODUCT_number PRODUCT_INTRODUCTION_DATE_introduced  \
0              1                            15-2-2011   
1             10                            15-2-2011   
2            100                            15-2-2011   
3            101                           15-12-2019   
4            102                           10-12-2019   

  PRODUCT_PRODUCT_TYPE_code PRODUCT_PRODUCTION_COST_cost  \
0                         1                            4   
1                         1                           10   
2                        17                            3   
3                        18                       305.54   
4                        18                       380.95   

  PRODUCT_MARGIN_margin PRODUCT_image_image PRODUCT_LANGUAGE_language  \
0                   .33       P01CE1CG1.jpg                        EN   
1                    .4       P10CE1CG1.jpg                        EN   
2                    .5     P100OP4FA17.jpg                        EN   
3   

### Order_details

In [24]:
order_details = go_sales_order_details

# replacing original columns with unit and returned_item
order_details['ORDER_DETAILS_UNIT_ID_unit'] = unit.index

# merging the returned_item and order_details
order_details = pd.merge(order_details, returned_item, left_on='ORDER_DETAIL_CODE', right_on='RETURNED_ITEM_ORDER_DETAIL_CODE_detail_code')

# dropping the columns replaced by unit and returned_item
order_details = order_details.drop(columns=['UNIT_COST', 'UNIT_PRICE', 'UNIT_SALE_PRICE']) 


# dropping the trial879 column
order_details = order_details.drop(columns=['TRIAL879'])

# rename the columns
order_details = order_details.rename(columns={'ORDER_DETAILS_CODE': 'ORDER_DETAILS_code', 'QUANTITY': 'ORDER_DETAILS_QUANTITY_quantity', 'TOTAL_COST': 'ORDER_DETAILS_TOTAL_COST_total', 'TOTAL_MARGIN': 'ORDER_DETAILS_TOTAL_MARGIN_margin', 'RETURN_CODE': 'ORDER_DETAILS_RETURN_CODE_returned', 'ORDER_NUMBER': 'ORDER_DETAILS_ORDER_NUMBER_order', 'PRODUCT_NUMBER': 'ORDER_DETAILS_PRODUCT_NUMBER_product', 'UNIT_ID': 'ORDER_DETAILS_UNIT_ID_unit'})

print(order_details.head())

  ORDER_DETAIL_CODE ORDER_DETAILS_ORDER_NUMBER_order  \
0            100146                             8465   
1            100149                             9116   
2            100199                             9113   
3            100250                             9254   
4            100641                             9269   

  ORDER_DETAILS_PRODUCT_NUMBER_product  ORDER_DETAILS_QUANTITY_quantity  \
0                                   15                               16   
1                                   16                               64   
2                                   30                               22   
3                                   43                               52   
4                                  111                                6   

   ORDER_DETAILS_UNIT_ID_unit  ORDER_DETAILS_UNIT_ID_unit RETURNED_ITEM_code  \
0                         144                         144               1975   
1                         147                       

### Training

In [25]:
training = go_staff_training

# replacing the original columns with year and course
# merging the year and course
training = pd.merge(training, year, left_on='YEAR', right_on='YEAR')
training = pd.merge(training, course, left_on='COURSE_CODE', right_on='COURSE_code')

# dropping the trial633 column
training = training.drop(columns=['TRIAL633'])

# rename the columns
training = training.rename(columns={'SALES_STAFF_CODE': 'TRAINING_SALES_STAFF_CODE', 'COURSE_CODE': 'TRAINING_COURSE_CODE', 'YEAR': 'TRAINING_YEAR'})

print(training.head())


  TRAINING_YEAR TRAINING_SALES_STAFF_CODE TRAINING_COURSE_CODE COURSE_code  \
0          2004                       100                    2           2   
1          2004                       100                    3           3   
2          2004                       100                    4           4   
3          2005                       100                    5           5   
4          2006                       100                    6           6   

  COURSE_DESCRIPTION  
0   GO Communication  
1         GO Sales 1  
2         GO Sales 2  
3     GO Marketing 1  
4     GO Marketing 2  


### Satisfaction

In [26]:
satisfaction = go_staff_satisfaction

# replacing the original columns with year and satisfaction_type
# merging the year and satisfaction_type
satisfaction = pd.merge(satisfaction, year, left_on='YEAR', right_on='YEAR')
satisfaction = pd.merge(satisfaction, satisfaction_type, left_on='SATISFACTION_TYPE_CODE', right_on='SATISFACTION_TYPE_code')

# dropping the trial633 column
satisfaction = satisfaction.drop(columns=['TRIAL633'])

# rename the columns
satisfaction = satisfaction.rename(columns={'SALES_STAFF_CODE': 'SATISFACTION_SALES_STAFF_CODE', 'SATISFACTION_TYPE_CODE': 'SATISFACTION_SATISFACTION_TYPE_CODE', 'YEAR': 'SATISFACTION_YEAR'})

print(satisfaction.head())

  SATISFACTION_YEAR SATISFACTION_SALES_STAFF_CODE  \
0              2004                           100   
1              2004                           101   
2              2004                           102   
3              2004                           103   
4              2004                           104   

  SATISFACTION_SATISFACTION_TYPE_CODE SATISFACTION_TYPE_code  \
0                                   1                      1   
1                                   4                      4   
2                                   4                      4   
3                                   3                      3   
4                                   4                      4   

  SATISFACTION_TYPE_DESCRIPTION  
0                 Not satisfied  
1                Very Satisfied  
2                Very Satisfied  
3                     Satisfied  
4                Very Satisfied  


### Order_header

In [27]:
order_header = pd.merge(go_sales_order_header, retailer, left_on='RETAILER_NAME', right_on='COMPANY_NAME')

# converting the date to datetime
order_header['ORDER_DATE'] = pd.to_datetime(order_header['ORDER_DATE'])

# replacing the original columns with date, sales_staff, retailer_site and retailer_contact
# merging the date, sales_staff, retailer_site and retailer_contact
order_header = pd.merge(order_header, date, left_on='ORDER_DATE', right_on='DATE_date')
order_header = pd.merge(order_header, sales_staff, left_on='SALES_STAFF_CODE', right_on='SALES_STAFF_code')
order_header = pd.merge(order_header, retailer_site, left_on='RETAILER_SITE_CODE', right_on='RETAILER_SITE_code')
order_header = pd.merge(order_header, retailer_contact, left_on='RETAILER_CONTACT_CODE', right_on='RETAILER_CONTACT_code')

# dropping the trial885 column
order_header = order_header.drop(columns=['TRIAL885'])

# rename the columns
order_header = order_header.rename(columns={'ORDER_NUMBER': 'ORDER_HEADER_number', 'RETAILER_NAME': 'ORDER_HEADER_RETAILER_NAME', 'SALES_STAFF_CODE': 'ORDER_HEADER_SALES_STAFF', 'SALES_BRANCH_CODE': 'ORDER_HEADER_SALES_BRANCH_CODE', 'ORDER_DATE': 'ORDER_HEADER_ORDER_DATE', 'RETAILER_SITE_CODE': 'ORDER_HEADER_RETAILER_SITE_CODE', 'RETAILER_CONTACT_CODE': 'ORDER_HEADER_RETAILER_CONTACT_CODE'})

print(order_header.head())

  ORDER_HEADER_number ORDER_HEADER_RETAILER_NAME  \
0                1153               Ultra Sports   
1                1153               Ultra Sports   
2                1154               Ultra Sports   
3                1154               Ultra Sports   
4                1155               Ultra Sports   

  ORDER_HEADER_RETAILER_SITE_CODE ORDER_HEADER_RETAILER_CONTACT_CODE  \
0                               4                                  6   
1                               4                                  6   
2                               4                                  6   
3                               4                                  6   
4                               5                                 65   

  ORDER_HEADER_SALES_STAFF ORDER_HEADER_SALES_BRANCH_CODE  \
0                       50                             18   
1                       50                             18   
2                       50                             18   
3         

### GO_SALES_INVENTORY_LEVELS

In [28]:
go_sales_inventory_levels = go_sales_inventory_levels.rename(columns={'INVENTORY_COUNT': 'GO_SALES_INVENTORY_LEVELS_INVENTORY_COUNT', 'PRODUCT_NUMBER': 'GO_SALES_INVENTORY_LEVELS_PRODUCT_NUMBER', 'YEAR_MONTH': 'GO_SALES_INVENTORY_LEVELS_YEAR_MONTH'})

print(go_sales_inventory_levels.head())

   INVENTORY_YEAR  INVENTORY_MONTH  GO_SALES_INVENTORY_LEVELS_PRODUCT_NUMBER  \
0            2021                4                                        48   
1            2021                4                                        49   
2            2021                4                                        50   
3            2021                4                                        51   
4            2021                4                                        52   

   GO_SALES_INVENTORY_LEVELS_INVENTORY_COUNT  
0                                       1932  
1                                       1400  
2                                      21705  
3                                       9710  
4                                       5616  


### GO_SALES_PRODUCT_FORECAST

In [29]:
go_sales_product_forecast = go_sales_product_forcast.rename(columns={'EXPECTED_VOLUME': 'GO_SALES_PRODUCT_FORECAST_EXPECTED_VOLUME', 'EXPECTED_COST': 'GO_SALES_PRODUCT_FORECAST_EXPECTED_COST', 'EXPECTED_MARGIN': 'GO_SALES_PRODUCT_FORECAST_EXPECTED_MARGIN', 'PRODUCT_NUMBER': 'GO_SALES_PRODUCT_FORECAST_PRODUCT_NUMBER', 'YEAR_MONTH': 'GO_SALES_PRODUCT_FORECAST_YEAR_MONTH'})

print(go_sales_product_forecast.head())

   GO_SALES_PRODUCT_FORECAST_PRODUCT_NUMBER  YEAR  MONTH  \
0                                        44  2022     12   
1                                        45  2021      1   
2                                        45  2021      2   
3                                        45  2021      3   
4                                        45  2021      4   

   GO_SALES_PRODUCT_FORECAST_EXPECTED_VOLUME  
0                                        383  
1                                         80  
2                                         51  
3                                        214  
4                                        300  


# Omzetten van de data
Omzetten van de data naar de juiste data types

In [30]:
#TODO

# Toevoegen van nieuwe kolommen
Toevoegen van de afgeleide informatie die ook in het ETL en de data warehouse moet komen

In [31]:
#TODO